<a href="https://colab.research.google.com/github/mirana60/git_tutorial/blob/master/%E6%A0%AA%E4%BE%A1%E4%BA%88%E6%B8%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

df = pd.read_csv('/content/train.csv')
df_test_base = pd.read_csv('/content/test.csv')
df_submit = pd.read_csv('/content/sample_submission.csv')


def data_pre(df, col):
  df = pd.concat([df,df[col].diff(1).rename(f'{col}'),df[col].diff(1).shift(1).rename(f'{col}_1day_diff'),df[col].diff(1).shift(2).rename(f'{col}_2day_diff'),df[col].diff(1).shift(3).rename(f'{col}_3day_diff'),df[col].diff(1).shift(4).rename(f'{col}_4day_diff'),df[col].diff(1).shift(5).rename(f'{col}_5day_diff')],axis=1)
  df = df.drop(col, axis=1)
  return df

## 学習データへの特徴量追加
df = df.drop(["Date"], axis=1)
df = data_pre(df, "Open")
df = data_pre(df, "High")
df = data_pre(df, "Low")
df = data_pre(df, "Close")


## テストデータへの特徴量追加
df_test = df_test_base.drop(["Date"], axis=1)
df_test = data_pre(df_test, "Open")
df_test = data_pre(df_test, "High")
df_test = data_pre(df_test, "Low")
df_test = data_pre(df_test, "Close")



df_train, df_val = train_test_split(df,test_size=0.2)
col = 'Up'
X_train = df_train.drop([col],axis=1)
y_train = df_train[col]
X_val = df_val.drop([col],axis=1)
y_val = df_val[col]

trains = lgb.Dataset(X_train,y_train)
valids = lgb.Dataset(X_val,y_val)

params = {'objective':'binary',
          'metric':'binary_logloss'}

model = lgb.train(
    params,
    trains,
    valid_sets=[valids],  # 検証データをリストで渡す
    num_boost_round=1000,callbacks= [lgb.early_stopping(stopping_rounds=100) ])

predict = model.predict(df_test)
thershold = 0.5

predict_list = []
for i in predict:
  if i >= thershold:
    predict_list.append(1)
  else:
    predict_list.append(0)


df_submit = pd.merge(df_submit.iloc[:, [0]].reset_index(drop=True), df_predict.iloc[:, [0, 1]].reset_index(drop=True), how='left', left_index=True, right_index=True)

df_submit.to_csv('submit.csv', index=False)








[LightGBM] [Info] Number of positive: 840, number of negative: 780
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000509 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5100
[LightGBM] [Info] Number of data points in the train set: 1620, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.518519 -> initscore=0.074108
[LightGBM] [Info] Start training from score 0.074108
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[3]	valid_0's binary_logloss: 0.693557


In [ ]:
!pip install yfinance
!pip install plotly
!pip install lightgbm
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import lightgbm as lgb
import yfinance as yf
import matplotlib.pyplot as plt


ticker = 'GOOG'
period = '1y'
interval = '1d'
data_df = yf.download(ticker, period=period, interval=interval)
data_df


[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,GOOG,GOOG,GOOG,GOOG,GOOG
Date,,,,,
2024-01-18,144.470917,145.063788,142.836789,142.926463,18876800
2024-01-19,147.440231,147.509973,145.278002,145.781184,27170900
2024-01-22,147.181168,149.477908,147.051628,148.177587,21829200
2024-01-23,148.147675,148.327038,146.663019,147.191120,14113600
2024-01-24,149.811722,151.027355,149.303538,149.751924,19245000
...,...,...,...,...,...
2025-01-13,192.289993,192.490005,188.660004,191.350006,13169100
2025-01-14,191.050003,193.270004,189.639999,192.500000,13651200


In [ ]:
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import lightgbm as lgb
import yfinance as yf
import matplotlib.pyplot as plt

import plotly.io as pio
pio.renderers.default = 'colab'

ticker = 'GOOG'
period = '1y'
interval = '1d'
data_df = yf.download(ticker, period=period, interval=interval)


fig = go.Figure(data=[go.Candlestick(x=data_df.index,open=data_df['Open'],high=data_df['High'],low=data_df['Low'],close=data_df['Close'])])
fig.update_layout(title='Google株価',xaxis_title='日付',yaxis_title='株価')


fig.show()


[*********************100%***********************]  1 of 1 completed


In [ ]:

import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import lightgbm as lgb
import yfinance as yf
import matplotlib.pyplot as plt

import plotly.io as pio
pio.renderers.default = 'colab'


ticker = 'GOOG'
period = '1y'
interval = '1d'
data_df = yf.download(ticker, period=period, interval=interval)

fig = go.Figure(
    data=[
        go.Candlestick(
            x=data_df.index,
            open=data_df["Open"],
            high=data_df["High"],
            low=data_df["Low"],
            close=data_df["Close"],
        )
    ]
)

fig.update_layout(
    title="Google 株価",
    xaxis_title="日付",
    yaxis_title="株価（USD）"
)

fig.show()

[*********************100%***********************]  1 of 1 completed


In [ ]:
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import lightgbm as lgb
import yfinance as yf
import matplotlib.pyplot as plt


ticker = 'GOOG'
period = '1y'
interval = '1d'
data_df = yf.download(ticker, period=period, interval=interval)
data_df

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,GOOG,GOOG,GOOG,GOOG,GOOG
Date,,,,,
2024-01-18,144.470901,145.063772,142.836774,142.926448,18876800
2024-01-19,147.440216,147.509958,145.277987,145.781169,27170900
2024-01-22,147.181183,149.477923,147.051643,148.177603,21829200
2024-01-23,148.147690,148.327053,146.663034,147.191135,14113600
2024-01-24,149.811722,151.027355,149.303538,149.751924,19245000
...,...,...,...,...,...
2025-01-13,192.289993,192.490005,188.660004,191.350006,13169100
2025-01-14,191.050003,193.270004,189.639999,192.500000,13651200
